In [1]:
import os
import sys
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.agents import create_agent
from pathlib import Path

load_dotenv()
api_key=os.environ.get("GOOGLE_API_KEY")

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [2]:
root_path = str(Path(os.getcwd()).resolve().parent) 
if root_path not in sys.path:
    sys.path.append(root_path)

In [3]:
# Create the agent
from tools.retrieve_and_reply import fast_search_engine
from langchain.agents import create_agent

tools = [fast_search_engine]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from the financial report. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [5]:
query = (
    "What is the revenue of GOOG in three months ending 2025 Q3? Find it in the income statement."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the revenue of GOOG in three months ending 2025 Q3? Find it in the income statement.
================================== Ai Message ==================================
Tool Calls:
  fast_search_engine (9b5a181c-65e3-4d66-b082-52cb5bb2d8a1)
 Call ID: 9b5a181c-65e3-4d66-b082-52cb5bb2d8a1
  Args:
    query: revenue of GOOG in three months ending 2025 Q3 in the income statement
================================= Tool Message =================================
Name: fast_search_engine

Total revenues for GOOG in the three months ending September 30, 2025, were $102,346 million.
================================== Ai Message ==================================

[{'type': 'text', 'text': 'The revenue of GOOG in the three months ending 2025 Q3 was $102,346 million.', 'extras': {'signature': 'CpcCAXLI2nzRroWOuoLG4/wAG8GJ3TMyq8AvrYIGqhg9XpxPcHRSnD44CAVtp3WimiP8b75+F1/5KZzCWhsUYrIReVGc8I55w3ykQ1ei+i3IApMySikdtQq9+

Now that we have a basic Agent, we are shifting focus to two pillars: Memory Management and Context Engineering. The former ensures the Agent remembers the user's conversation history, while the latter refines the input context, providing the Agent with the necessary data to deliver precise and informed responses.

In [6]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig

checkpointer = InMemorySaver()

agent_memory = create_agent(
    model, 
    tools,
    middleware=[
        SummarizationMiddleware(
            model,
            trigger=("tokens", 4000),
            keep=("messages", 20)
        )
    ],
    checkpointer=checkpointer,
)

config: RunnableConfig = {"configurable": {"thread_id": "2"}}

In [7]:
queries = [
    "hi, my name is bob",
    "What is the R&D expense of Alphabet in three months ended 2025Q3?",
    "How about the share repurchase?",
    "What did I asked about Google in the last 2 conversations?"
]

responses = []

for q in queries:
    print(f"\n--- The question is: {q} ---")
    res = agent_memory.invoke({"messages": q}, config)
    responses.append(res)
    res["messages"][-1].pretty_print()


--- The question is: hi, my name is bob ---
================================== Ai Message ==================================

Hello Bob! How can I help you today?

--- The question is: What is the R&D expense of Alphabet in three months ended 2025Q3? ---
================================== Ai Message ==================================

[{'type': 'text', 'text': 'Research and development expenses for the three months ended September 30, 2025 were $15,151 million.', 'extras': {'signature': 'CpMDAXLI2nzviAHQDbOWjzCZ2gYrGzfIC1QLXFaerbBuFAEqrNlLeVZ+o4qv+oIem7fvt7xAF3e94JWeFmUDPBFH+TXTcvL/wOOiNuyIKrjEBxvX1JNAmKfP7KFK6rP3YtCphmLbFxp5RORO4EpktqVtjANvnmwn98c69tiFpFr0EJgD16r7ZLYqX2i0r94DbKlZJ/st1rVyHMS3w4HYn2Hh1HGlN5lGt8RivyY3X0tgqUZERN/HLSceuyDzYetsBDk1li6vzP7ckWB6jfyOzhtnYi2weR06c/fzkfFdmWajPW/FYUTqZEiOzXzSqntOnTsiaGpVXo6kvz2EVfW7kFsb/OtxNyaSb8eCiD7XSF8ShCHETBUCdhc2aEUdsTEgHwqoOaULHSgINEbKlFSCVcCgtOplxvb31rJL1ly9RO7O81b/6pUwHLSwBY/08F7cNCX/G67NiwbwqQqhtAk0iCk2flRwelKXayjdnyKYkX3iQZeo8oyUfnulaz